In [0]:
from pyspark.sql.functions import spark_partition_id
data1 = [{'Name':'Jhon','ID':21.528,'Add':'USA'},{'Name':'Joe','ID':3.69,'Add':'USA'},{'Name':'Tina','ID':2.48,'Add':'IND'},{'Name':'Jhon','ID':22.22, 'Add':'USA'},{'Name':'Joe','ID':5.33,'Add':'INA'}]
a = sc.parallelize(data1)
b = spark.createDataFrame(a)
b=b.withColumn("partitionID", spark_partition_id())
b.show()

+---+------+----+-----------+
|Add|    ID|Name|partitionID|
+---+------+----+-----------+
|USA|21.528|Jhon|          0|
|USA|  3.69| Joe|          1|
|IND|  2.48|Tina|          2|
|USA| 22.22|Jhon|          3|
|INA|  5.33| Joe|          3|
+---+------+----+-----------+



In [0]:
data2 = [{'Name':'Atin','ID':21.528,'Add':'India'},{'Name':'Joe','ID':3.69,'Add':'USeA'},{'Name':'Tina','ID':2.48,'Add':'IND'},{'Name':'Jhon','ID':22.22, 'Add':'USdA'},{'Name':'Joe','ID':5.33,'Add':'rsa'}]
c = sc.parallelize(data2)
d = spark.createDataFrame(c)
d=d.withColumn("partitionID", spark_partition_id())
d.show()

+-----+------+----+-----------+
|  Add|    ID|Name|partitionID|
+-----+------+----+-----------+
|India|21.528|Atin|          0|
| USeA|  3.69| Joe|          1|
|  IND|  2.48|Tina|          2|
| USdA| 22.22|Jhon|          3|
|  rsa|  5.33| Joe|          3|
+-----+------+----+-----------+



In [0]:
f = d.join(b,d.Add == b.Add)
f.explain(True)

== Parsed Logical Plan ==
Join Inner, (Add#4315 = Add#4287)
:- Project [Add#4315, ID#4316, Name#4317, SPARK_PARTITION_ID() AS partitionID#4321]
:  +- LogicalRDD [Add#4315, ID#4316, Name#4317], false
+- Project [Add#4287, ID#4288, Name#4289, SPARK_PARTITION_ID() AS partitionID#4293]
   +- LogicalRDD [Add#4287, ID#4288, Name#4289], false

== Analyzed Logical Plan ==
Add: string, ID: double, Name: string, partitionID: int, Add: string, ID: double, Name: string, partitionID: int
Join Inner, (Add#4315 = Add#4287)
:- Project [Add#4315, ID#4316, Name#4317, SPARK_PARTITION_ID() AS partitionID#4321]
:  +- LogicalRDD [Add#4315, ID#4316, Name#4317], false
+- Project [Add#4287, ID#4288, Name#4289, SPARK_PARTITION_ID() AS partitionID#4293]
   +- LogicalRDD [Add#4287, ID#4288, Name#4289], false

== Optimized Logical Plan ==
Join Inner, (Add#4315 = Add#4287)
:- Filter isnotnull(Add#4315)
:  +- Project [Add#4315, ID#4316, Name#4317, SPARK_PARTITION_ID() AS partitionID#4321]
:     +- LogicalRDD [Add#43

In [0]:
f.show()

+---+----+----+-----------+---+----+----+-----------+
|Add|  ID|Name|partitionID|Add|  ID|Name|partitionID|
+---+----+----+-----------+---+----+----+-----------+
|IND|2.48|Tina|          2|IND|2.48|Tina|          2|
+---+----+----+-----------+---+----+----+-----------+



In [0]:
from pyspark.sql.functions import broadcast
f = d.join(broadcast(b),d.Add == b.Add)
f.explain(True)

== Parsed Logical Plan ==
Join Inner, (Add#4315 = Add#4287)
:- Project [Add#4315, ID#4316, Name#4317, SPARK_PARTITION_ID() AS partitionID#4321]
:  +- LogicalRDD [Add#4315, ID#4316, Name#4317], false
+- ResolvedHint (strategy=broadcast)
   +- Project [Add#4287, ID#4288, Name#4289, SPARK_PARTITION_ID() AS partitionID#4293]
      +- LogicalRDD [Add#4287, ID#4288, Name#4289], false

== Analyzed Logical Plan ==
Add: string, ID: double, Name: string, partitionID: int, Add: string, ID: double, Name: string, partitionID: int
Join Inner, (Add#4315 = Add#4287)
:- Project [Add#4315, ID#4316, Name#4317, SPARK_PARTITION_ID() AS partitionID#4321]
:  +- LogicalRDD [Add#4315, ID#4316, Name#4317], false
+- ResolvedHint (strategy=broadcast)
   +- Project [Add#4287, ID#4288, Name#4289, SPARK_PARTITION_ID() AS partitionID#4293]
      +- LogicalRDD [Add#4287, ID#4288, Name#4289], false

== Optimized Logical Plan ==
Join Inner, (Add#4315 = Add#4287), rightHint=(strategy=broadcast), joinId=3
:- Filter isnotn

In [0]:
f.show()

+---+----+----+-----------+---+----+----+-----------+
|Add|  ID|Name|partitionID|Add|  ID|Name|partitionID|
+---+----+----+-----------+---+----+----+-----------+
|IND|2.48|Tina|          2|IND|2.48|Tina|          2|
+---+----+----+-----------+---+----+----+-----------+



In [0]:
f.rdd.getNumPartitions()

4

In [0]:
peopleDF = (
  ("andrea", "medellin"),
  ("rodolfo", "medellin"),
  ("abdul", "bangalore")
)

peopleDF_a = sc.parallelize(peopleDF)
peopleDF_b = spark.createDataFrame(peopleDF_a, ["first_name", "city"])

peopleDF_b.show()

+----------+---------+
|first_name|     city|
+----------+---------+
|    andrea| medellin|
|   rodolfo| medellin|
|     abdul|bangalore|
+----------+---------+



In [0]:
citiesDF = (
  ("medellin", "colombia", 2.5),
  ("bangalore", "india", 12.3)
)




citiesDF_a = sc.parallelize(citiesDF)
citiesDF_b = spark.createDataFrame(citiesDF_a, ("city", "country", "population"))

citiesDF_b.show()

+---------+--------+----------+
|     city| country|population|
+---------+--------+----------+
| medellin|colombia|       2.5|
|bangalore|   india|      12.3|
+---------+--------+----------+



In [0]:
peopleDF_b_j = peopleDF_b.join(
  broadcast(citiesDF_b),
  peopleDF_b.city == citiesDF_b.city
)

peopleDF_b_j.explain(True)

== Parsed Logical Plan ==
Join Inner, (city#4454 = city#4466)
:- LogicalRDD [first_name#4453, city#4454], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [city#4466, country#4467, population#4468], false

== Analyzed Logical Plan ==
first_name: string, city: string, city: string, country: string, population: double
Join Inner, (city#4454 = city#4466)
:- LogicalRDD [first_name#4453, city#4454], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [city#4466, country#4467, population#4468], false

== Optimized Logical Plan ==
Join Inner, (city#4454 = city#4466), rightHint=(strategy=broadcast), joinId=5
:- Filter isnotnull(city#4454)
:  +- LogicalRDD [first_name#4453, city#4454], false
+- Filter isnotnull(city#4466)
   +- LogicalRDD [city#4466, country#4467, population#4468], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
   BroadcastHashJoin [city#4454], [city#4466], Inner, BuildRight, false, true
   :- Filter isnotnull(city#4454)


In [0]:
peopleDF_b_j = peopleDF_b.join(
  citiesDF_b,
  peopleDF_b.city == citiesDF_b.city
)

peopleDF_b_j.explain(True)

== Parsed Logical Plan ==
Join Inner, (city#4454 = city#4466)
:- LogicalRDD [first_name#4453, city#4454], false
+- LogicalRDD [city#4466, country#4467, population#4468], false

== Analyzed Logical Plan ==
first_name: string, city: string, city: string, country: string, population: double
Join Inner, (city#4454 = city#4466)
:- LogicalRDD [first_name#4453, city#4454], false
+- LogicalRDD [city#4466, country#4467, population#4468], false

== Optimized Logical Plan ==
Join Inner, (city#4454 = city#4466)
:- Filter isnotnull(city#4454)
:  +- LogicalRDD [first_name#4453, city#4454], false
+- Filter isnotnull(city#4466)
   +- LogicalRDD [city#4466, country#4467, population#4468], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
   SortMergeJoin [city#4454], [city#4466], Inner
   :- Sort [city#4454 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(city#4454, 200), ENSURE_REQUIREMENTS, [plan_id=2055]
   :     +- Filter isnotnull(city#4454)
   :    

#### Notice how the parsed, analyzed, and optimized logical plans all contain ResolvedHint isBroadcastable=true because the broadcast() function was used. This hint isn’t included when the broadcast() function isn’t used.

## Automatic Detection

- In many cases, Spark can automatically detect whether to use a broadcast join or not, depending on the size of the data. If Spark can detect that one of the joined DataFrames is small (10 MB by default), Spark will automatically broadcast it for us. The code below:

In [0]:
bigTable = spark.range(1, 100000000)
smallTable = spark.range(1, 10000) # size estimated by Spark - auto-broadcast
joinedNumbers = smallTable.join(bigTable, "id")
joinedNumbers.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner, [id])
:- Range (1, 10000, step=1, splits=Some(4))
+- Range (1, 100000000, step=1, splits=Some(4))

== Analyzed Logical Plan ==
id: bigint
Project [id#4511L]
+- Join Inner, (id#4511L = id#4509L)
   :- Range (1, 10000, step=1, splits=Some(4))
   +- Range (1, 100000000, step=1, splits=Some(4))

== Optimized Logical Plan ==
Project [id#4511L]
+- Join Inner, (id#4511L = id#4509L)
   :- Range (1, 10000, step=1, splits=Some(4))
   +- Range (1, 100000000, step=1, splits=Some(4))

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- == Initial Plan ==
   ColumnarToRow
   +- PhotonResultStage
      +- PhotonProject [id#4511L]
         +- PhotonBroadcastHashJoin [id#4511L], [id#4509L], Inner, BuildLeft, false, true
            :- PhotonShuffleExchangeSource
            :  +- PhotonShuffleMapStage
            :     +- PhotonShuffleExchangeSink SinglePartition
            :        +- PhotonRange Range (1, 10000, step=1, splits=4)
            +-

- However, in the previous case, Spark did not detect that the small table could be broadcast. 
- The reason is that Spark will not determine the size of a local collection because it might be big, and evaluating its size may be an O(N) operation, which can defeat the purpose before any computation is made.

- Spark will perform auto-detection when
    - it constructs a DataFrame from scratch, e.g. spark.range
    - it reads from files with schema and/or size information, e.g. Parquet

## Configuring Broadcast Join Detection

- The threshold for automatic broadcast join detection can be tuned or disabled.
- The configuration is spark.sql.autoBroadcastJoinThreshold, and the value is taken in bytes. If you want to configure it to another number, we can set it in the SparkSession:

In [0]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 104857600)

- or deactivate it altogether by setting the value to -1.

In [0]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1) 